In [58]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import soundfile
from numpy.lib.stride_tricks import as_strided

In [15]:
tf.test.is_gpu_available()

False

In [16]:
# with tf.device('/device:CPU:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
#     b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
#     c = tf.matmul(a, b)

# with tf.Session() as sess:
#     print(sess.run(a))

In [ ]:
def spectrogram(samples, fft_len, sample_rate=2, hop_length=128):
    window = np.hanning(fft_len)[:, None]
    window_norm = np.sum(window**2)
    scale = window_norm * sample_rate
    trunc = (len(samples) - fft_len) % hop_length
    x = samples[:len(samples) - trunc]
    nshape = (fft_len, (len(x) - fft_len) // hop_length + 1)
    nstrides = (x.strides[0], x.strides[0] * hop_length)
    x = as_strided(x, shape=nshape, strides=nstrides)
    
    x = np.fft.rfft(x * window, axis=0)
    x = np.absolute(x)**2
    x[1:-1, :] *= (2.0 / scale)
    freqs = float(sample_rate) / fft_len * np.arange(x.shape[0])
    return x, freqs
    

In [40]:
def spectrogram_audio(filename, step=10, window=20, max_freq=None, eps=1e-14):
    with soundfile.SoundFile(filename) as sf:
        audio = sf.read(dtype='float32')
        sample_rate = sf.samplerate
        if audio.ndim >= 2:
            audio = np.mean(audio, 1)
        if max_freq is None:
            max_freq = sample_rate / 2
        if max_freq > sample_rate / 2:
            raise Valueerror("max freq canot be > than 0.5 of sample rate")
        hop_length = int(0.001 * step * sample_rate)
        fft_length = int(0.001 * window * sample_rate)
        pxx, freqs = spectrogram(audio, fft_length=fft_length, 
                         sample_rate=sample_rate, hop_length=hop_length)
        ind = np.where(freqs <= max_freq)[0][-1] + 1
    return np.transpose(np.log(pxx[:ind], :) + eps)


In [41]:
spectrogram_audio('/home/jugs/Desktop/p257_430.wav')

48000


AttributeError: 'numpy.ndarray' object has no attribute 'summary'

In [63]:
han = np.hanning(5)[:, None]
print(han)
win = np.sum(han**2)
print(win)

[[0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]]
1.5
